In [1]:
import wat

In [2]:
import pathlib

data_dir = pathlib.Path("../data/")
document_paths = {
    'journal' : data_dir / 'journal' / 'JOURNAL_2017_AerationCostsInStirredTankAndBubbleColumnBioreactors.pdf',
    'textbook' : data_dir / 'textbook' / 'TEXT_DairyProcessingHandbook_WheyProcessingChapter15.pdf',
    'report' : data_dir / 'report' / 'REPORT_ConsultancyOnLargeScaleSubmergedAerobicCultivationProcessDesignNRELGenomatica.pdf',
    'article': data_dir / 'article' / 'REPORT_2003_OptimizePowerConsumptionInAerobicFermenters.pdf'
}
document_paths = {
    k : str(v.resolve())
    for k,v in document_paths.items()
}

In [3]:
from pyzerox import zerox
import os
import json
import asyncio

### Model Setup (Use only Vision Models) Refer: https://docs.litellm.ai/docs/providers ###

## placeholder for additional model kwargs which might be required for some models
kwargs = {}

## system prompt to use for the vision model
custom_system_prompt = None

# to override
# custom_system_prompt = "For the below pdf page, do something..something..." ## example

###################### Supported Models and ENV vars ######################
# For other providers refer: https://docs.litellm.ai/docs/providers 
models = {
    'openai' : "gpt-4o-mini", #OPENAI_API_KEY
    'gemini' : "gemini/gemini-1.5-flash", #GEMINI_API_KEY
    'anthropic' : "claude-3-5-sonnet-20241022", #ANTHROPIC_API_KEY
    'haiku' : 'claude-3-haiku-20240307',
    'sonnet' : 'claude-3-5-sonnet-20241022',
}

model = models['gemini']

###### Define main async entrypoint
async def extract_markdown(filepath, provider: str = 'haiku'):
    if provider in models.keys():
        model=models[provider]
    else:
        raise KeyError(f"provider not supported, choose from {list(models.keys())}")
    
    # KWARGS docs: https://docs.litellm.ai/docs/completion/input
    
    ## local filepath and file URL supported
    file_path = str(pathlib.Path(filepath).resolve())

    ## process only some pages or all
    select_pages = None ## None for all, but could be int or list(int) page numbers (1 indexed)

    output_dir = "./output_test" ## directory to save the consolidated markdown file
    result = await zerox(
        file_path=file_path, 
        model=model, 
        output_dir=None,
        custom_system_prompt=custom_system_prompt,
        select_pages=select_pages, 
        **kwargs
    )
    return result




/Users/nicholasgrundl/projects/ragnostic/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [ ]:
# run the main function:
document_path = document_paths['article']
result = await extract_markdown(filepath=document_path)

# print markdown result
print(result.pages[1].content)

# Extract all of them

## Anthropic haiku

In [58]:
import pymupdf  # import package PyMuPDF
import time

def get_elapsed_time(start, end):
    seconds = end - start
    minutes = round(seconds/60,2)
    return seconds, minutes

provider='haiku'
time_log = {}
time_log['total'] = {
    'start' : time.time()
}
results = {}
for doc_kind, path in document_paths.items():
    start = time.time()
    print(f"{doc_kind} : {path}")
    
    doc_result = {}
    doc=pymupdf.open(path)
    print(f"- page count: {doc.page_count}")
    try:
        result = await extract_markdown(filepath=path, provider=provider)
        doc_result['result'] = result
    except Exception as e:
        print(f"- failed to extract")
        doc_result['error'] = e
    end = time.time()
    elapsed_s, elapsed_min = get_elapsed_time(start, end)
    time_log['doc_kind'] = {
        'start' : start,
        'end' : end,
        'elapsed' : elapsed_min,
    }
    doc_result['minutes'] = elapsed_min
    results[doc_kind] = doc_result
time_log['total']['end'] = time.time()
elapsed_s, elapsed_min = get_elapsed_time(start=time_log['total']['start'], end=time_log['total']['end'])
time_log['total']['elapsed'] = elapsed_min
display(time_log)


def pages_to_markdown(pages: list, output_file: str = "output.md"):
    markdown_string = ''
    with open(output_file, "w") as f:
        for page in pages:
            # Add page number as header
            page_header = f"## Page {page.page}\n\n"
            markdown_string+=page_header
            f.write(page_header)
            
            # Add content with proper markdown line breaks
            content = page.content.replace("\n", "  \n")
            markdown_string+=content
            f.write(content)
            
            # page separator
            separator = "\n\n---\n\n"
            markdown_string+=separator
            f.write(separator) 
    return markdown_string

for doc_kind, doc_result in results.items():
    r = doc_result.get('result',None)
    if r is None:
        continue
    pages = r.pages
    markdown = pages_to_markdown(pages=pages, output_file=f"{doc_kind}-{provider}.md")

18:51:24 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


journal : /Users/nicholasgrundl/projects/ragnostic/data/journal/JOURNAL_2017_AerationCostsInStirredTankAndBubbleColumnBioreactors.pdf
- page count: 6
SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'max_tokens': 10}
_is_function_call: False
RAW RESPONSE:
{"id":"msg_01XMjgY5vTtjahoE1sgZecyN","type":"message","role":"assistant","model":"claude-3-haiku-20240307","content":[{"type":"text","text":"I'm doing well, thanks for asking! As"}],"stop_reason":"max_tokens","stop_sequence":null,"usage":{"input_tokens":14,"cache_creation_input_tokens":0,"cache_read_input_tokens":0,"output_tokens":10}}




18:51:25 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
18:51:25 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
18:51:25 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
18:51:25 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
18:51:25 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
18:51:25 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): None
Final returned optional params: {}
_is_function_call: False
RAW RESPONSE:
<coroutine object AnthropicChatCompletion.acompletion_function at 0x131ae70b0>


ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): None
Final returned optional params: {}
_is_function_call: False
RAW RESPONSE:
<coroutine object AnthropicChatCompletion.acompletion_function at 0x131ae7680>


ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): None
Final returned optional params: {}
_is_function_call: False
RAW RESPONSE:
<coroutine object AnthropicChatCompletion.acompletion_function at 0x131ae6ae0>


ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): None
Final returned optional params: {}
_is_function_call: False
RAW RESPONSE:
<coroutine object AnthropicChatCompletion.acompletion_function at 0x131ae7870>


ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): 

18:51:26 - LiteLLM:WARNING: logging_utils.py:113 - `logging_obj` not found - unable to track `llm_api_duration_ms
ERROR:root:Failed to process image Error:
    Error in Completion Response. Error: litellm.RateLimitError: AnthropicException - {"type":"error","error":{"type":"rate_limit_error","message":"This request would exceed your organization’s rate limit of 10,000 output tokens per minute. For details, refer to: https://docs.anthropic.com/en/api/rate-limits; see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}}
    Please check the status of your model provider API status.
    
18:51:26 - LiteLLM:WARNING: logging_utils.py:113 - `logging_obj` not found - unable to track `llm_api_duration_ms
ERROR:root:Failed to process image Error:
    Error in Completion Response. Error: litellm.RateLim

...

18:51:54 - LiteLLM:WARNING: logging_utils.py:113 - `logging_obj` not found - unable to track `llm_api_duration_ms


RAW RESPONSE:
{"id":"msg_01LiHNaxmGCHhpa4JSE9UJGf","type":"message","role":"assistant","model":"claude-3-haiku-20240307","content":[{"type":"text","text":"# Reactions and Separations\n\n## Step 5. Calculate the required k₂a.\n## Step 6. Calculate the actual volumetric air flow at the agitator impeller tip. Use the volumetric flow rate to account for temperature, composition, backpressure and liquid head. Use the average flowrates for the inlet and outlet. \n## Step 7. Calculate the superficial gas velocity at the impeller using the above as volumetric.\n## Step 8. Using the mass-transfer relationship developed from pilot scale data, solve for the agitator power. Add about 5% to account for the gear drive and seal losses to get the required motor power draw.\n## Step 9. Calculate compressor power. Include the backstage, liquid head, pressure, and losses from the sparging, piping and filtration systems. Also include the manufacturer's compression efficiency to calculate motor power draw 

18:51:58 - LiteLLM:WARNING: logging_utils.py:113 - `logging_obj` not found - unable to track `llm_api_duration_ms


RAW RESPONSE:
{"id":"msg_01X9f3tmUUzK6qkoMGr8Stw1","type":"message","role":"assistant","model":"claude-3-haiku-20240307","content":[{"type":"text","text":"# Step 2. Normally, the iteration process would start at a value 20% higher than the literature value, but for this case, it was started at 0.02, 0.06, and 0.06, respectively, based on units of g, kW/m^3, and kw/m^2. This strategy was used to converge faster. The remaining steps in this process should be derived from values stated in the actual broth and impeller system data that will be used. Based on a required k_a of 0.0649/s and a superficial gas velocity of 0.0204 m/s, the required agitator power/V = 349 W/m^3. For a batch size of 114 M^3, this gives a total installed power of 39.8 kW. Allowing for 70% power transmission efficiency through the gear drive and seal, the motor power draw required is about 41.9 kW. This figure seems low, it is because the required OTR for this problem is low, so the \"easy\" fermentation (step 9) to

{'total': {'start': 1737687084.4622562,
  'end': 1737687118.2099662,
  'elapsed': 0.56},
 'doc_kind': {'start': 1737687107.529393,
  'end': 1737687118.209742,
  'elapsed': 0.18}}

## Open AI gpt4-mini

In [59]:
import pymupdf  # import package PyMuPDF
import time

def get_elapsed_time(start, end):
    seconds = end - start
    minutes = round(seconds/60,2)
    return seconds, minutes

provider='openai'
time_log = {}
time_log['total'] = {
    'start' : time.time()
}
results = {}
for doc_kind, path in document_paths.items():
    start = time.time()
    print(f"{doc_kind} : {path}")
    
    doc_result = {}
    doc=pymupdf.open(path)
    print(f"- page count: {doc.page_count}")
    try:
        result = await extract_markdown(filepath=path, provider=provider)
        doc_result['result'] = result
    except Exception as e:
        print(f"- failed to extract")
        doc_result['error'] = e
    end = time.time()
    elapsed_s, elapsed_min = get_elapsed_time(start, end)
    time_log['doc_kind'] = {
        'start' : start,
        'end' : end,
        'elapsed' : elapsed_min,
    }
    doc_result['minutes'] = elapsed_min
    results[doc_kind] = doc_result
time_log['total']['end'] = time.time()
elapsed_s, elapsed_min = get_elapsed_time(start=time_log['total']['start'], end=time_log['total']['end'])
time_log['total']['elapsed'] = elapsed_min
display(time_log)


def pages_to_markdown(pages: list, output_file: str = "output.md"):
    markdown_string = ''
    with open(output_file, "w") as f:
        for page in pages:
            # Add page number as header
            page_header = f"## Page {page.page}\n\n"
            markdown_string+=page_header
            f.write(page_header)
            
            # Add content with proper markdown line breaks
            content = page.content.replace("\n", "  \n")
            markdown_string+=content
            f.write(content)
            
            # page separator
            separator = "\n\n---\n\n"
            markdown_string+=separator
            f.write(separator) 
    return markdown_string

for doc_kind, doc_result in results.items():
    r = doc_result.get('result',None)
    if r is None:
        continue
    pages = r.pages
    markdown = pages_to_markdown(pages=pages, output_file=f"{doc_kind}-{provider}.md")

18:52:10 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


journal : /Users/nicholasgrundl/projects/ragnostic/data/journal/JOURNAL_2017_AerationCostsInStirredTankAndBubbleColumnBioreactors.pdf
- page count: 6
SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'max_tokens': 10, 'extra_body': {}}
RAW RESPONSE:
{"id": "chatcmpl-At45ip7p0cDcorlMo9wSqNT3UJhOe", "choices": [{"finish_reason": "length", "index": 0, "logprobs": null, "message": {"content": "I'm just a program, but I'm here and ready", "refusal": null, "role": "assistant", "audio": null, "function_call": null, "tool_calls": null}}], "created": 1737687130, "model": "gpt-4o-mini-2024-07-18", "object": "chat.completion", "service_tier": "default", "system_fingerprint": "fp_72ed7ab54c", "usage": {"completion_tokens": 10, "prompt_tokens": 14, "total_tokens": 24, "completion_tokens_details": {"accepted_prediction_tokens": 0, "audio_tokens": 0, "reasoning_tokens": 0, "rejected_prediction_tokens": 0}, "prompt_tokens_details"

18:52:11 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
18:52:11 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
18:52:11 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
18:52:11 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
18:52:11 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
18:52:11 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): None
Final returned optional params: {'extra_body': {}}
ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): None
Final returned optional params: {'extra_body': {}}
ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): None
Final returned optional params: {'extra_body': {}}
ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): None
Final returned optional params: {'extra_body': {}}
ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): None
Final returned optional params: {'extra_body': {}}
ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): None
Final returned optional params: {'extra_body': {}}
RAW RESPONSE:
{"id": "chatcmpl-At45lRzPfoJV7oVHpQExmPkEyKPKi", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "```markdown\n3. Results and discussion\n\n3.1. Bioreactor capital costs\n\nTa

18:52:47 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


...

18:54:11 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{"id": "chatcmpl-At47NHtWA0doDSol2jqjktbICYej2", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "```markdown\n## References\n\n1. National Renewable Energy Laboratory, Statement of Work, \u201cConsultancy on Large-Scale Submerged Aerobic Cultivation Process Design,\u201d Dec 21, 2015.\n2. D. Humbird, R. Davis, J.D. McMillan, Aeration Costs in Stirred-Tank and Bubble Column Bioreactors, Biochemical Engineering Journal, submitted.\n3. K. van 't Riet, J. Tramper, in Basic Bioreactor Design, New York: Marcel Dekker, Inc., 1991, ch. 2.3, pp. 245-250, 294.\n4. K. van 't Riet, R. G. J. M. van der Laans, \u201cMixing in bioreactor vessels,\u201d in Comprehensive Biotechnology, 2nd ed., Amsterdam, Netherlands: Elsevier, 2011, ch. 2.07, pp. 63-80.\n5. Online source: https://en.wikipedia.org/wiki/Monod_equation\n6. J. J. Heijnen, \u201cThe process reaction for bioprocess design: a thermodynamic approach,\u201d presented at the Advanced Cou

18:54:12 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
18:54:12 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
18:54:12 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.
18:54:12 - LiteLLM:WARNING: utils.py:381 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): None
Final returned optional params: {'extra_body': {}}
ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): None
Final returned optional params: {'extra_body': {}}
ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): None
Final returned optional params: {'extra_body': {}}
ASYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache'): None
Final returned optional params: {'extra_body': {}}
RAW RESPONSE:
{"id": "chatcmpl-At47hDIOMvyGHFyBpE451zrRuY7LG", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "```markdown\nReactions and Separations\n\n# Optimize Power Consumption in Aerobic Fermenters\n\n**GREGORY T. BENZ**  \nBenz Technology International, Inc.\n\nBy performing the necessary pilot work, and rigorously calculating the full-scale performance instead of using simple rules-of-thumb for scale-up, significant energy savings c

{'total': {'start': 1737687130.017079,
  'end': 1737687287.782578,
  'elapsed': 2.63},
 'doc_kind': {'start': 1737687251.0485508,
  'end': 1737687287.782393,
  'elapsed': 0.61}}

## Gemini billed

In [4]:
import pymupdf  # import package PyMuPDF
import time

def get_elapsed_time(start, end):
    seconds = end - start
    minutes = round(seconds/60,2)
    return seconds, minutes

provider='gemini'
time_log = {}
time_log['total'] = {
    'start' : time.time()
}
results = {}
for doc_kind, path in document_paths.items():
    start = time.time()
    print(f"{doc_kind} : {path}")
    
    doc_result = {}
    doc=pymupdf.open(path)
    print(f"- page count: {doc.page_count}")
    try:
        result = await extract_markdown(filepath=path, provider=provider)
        doc_result['result'] = result
    except Exception as e:
        print(f"- failed to extract")
        doc_result['error'] = e
    end = time.time()
    elapsed_s, elapsed_min = get_elapsed_time(start, end)
    time_log['doc_kind'] = {
        'start' : start,
        'end' : end,
        'elapsed' : elapsed_min,
    }
    doc_result['minutes'] = elapsed_min
    results[doc_kind] = doc_result
time_log['total']['end'] = time.time()
elapsed_s, elapsed_min = get_elapsed_time(start=time_log['total']['start'], end=time_log['total']['end'])
time_log['total']['elapsed'] = elapsed_min
display(time_log)


def pages_to_markdown(pages: list, output_file: str = "output.md"):
    markdown_string = ''
    with open(output_file, "w") as f:
        for page in pages:
            # Add page number as header
            page_header = f"## Page {page.page}\n\n"
            markdown_string+=page_header
            f.write(page_header)
            
            # Add content with proper markdown line breaks
            content = page.content.replace("\n", "  \n")
            markdown_string+=content
            f.write(content)
            
            # page separator
            separator = "\n\n---\n\n"
            markdown_string+=separator
            f.write(separator) 
    return markdown_string

for doc_kind, doc_result in results.items():
    r = doc_result.get('result',None)
    if r is None:
        continue
    pages = r.pages
    markdown = pages_to_markdown(pages=pages, output_file=f"{doc_kind}-{provider}.md")

journal : /Users/nicholasgrundl/projects/ragnostic/data/journal/JOURNAL_2017_AerationCostsInStirredTankAndBubbleColumnBioreactors.pdf
- page count: 6


18:58:03 - LiteLLM:WARNING: transformation.py:187 - No text in user content. Adding a blank text to user content, to ensure Gemini doesn't fail the request. Relevant Issue - https://github.com/BerriAI/litellm/issues/5515
18:58:03 - LiteLLM:WARNING: transformation.py:187 - No text in user content. Adding a blank text to user content, to ensure Gemini doesn't fail the request. Relevant Issue - https://github.com/BerriAI/litellm/issues/5515
18:58:03 - LiteLLM:WARNING: transformation.py:187 - No text in user content. Adding a blank text to user content, to ensure Gemini doesn't fail the request. Relevant Issue - https://github.com/BerriAI/litellm/issues/5515
18:58:03 - LiteLLM:WARNING: transformation.py:187 - No text in user content. Adding a blank text to user content, to ensure Gemini doesn't fail the request. Relevant Issue - https://github.com/BerriAI/litellm/issues/5515
18:58:03 - LiteLLM:WARNING: transformation.py:187 - No text in user content. Adding a blank text to user content, to

textbook : /Users/nicholasgrundl/projects/ragnostic/data/textbook/TEXT_DairyProcessingHandbook_WheyProcessingChapter15.pdf
- page count: 22


18:58:24 - LiteLLM:WARNING: transformation.py:187 - No text in user content. Adding a blank text to user content, to ensure Gemini doesn't fail the request. Relevant Issue - https://github.com/BerriAI/litellm/issues/5515
18:58:24 - LiteLLM:WARNING: transformation.py:187 - No text in user content. Adding a blank text to user content, to ensure Gemini doesn't fail the request. Relevant Issue - https://github.com/BerriAI/litellm/issues/5515
18:58:24 - LiteLLM:WARNING: transformation.py:187 - No text in user content. Adding a blank text to user content, to ensure Gemini doesn't fail the request. Relevant Issue - https://github.com/BerriAI/litellm/issues/5515
18:58:24 - LiteLLM:WARNING: transformation.py:187 - No text in user content. Adding a blank text to user content, to ensure Gemini doesn't fail the request. Relevant Issue - https://github.com/BerriAI/litellm/issues/5515
18:58:24 - LiteLLM:WARNING: transformation.py:187 - No text in user content. Adding a blank text to user content, to


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



...

18:58:26 - LiteLLM:WARNING: logging_utils.py:113 - `logging_obj` not found - unable to track `llm_api_duration_ms
ERROR:root:Failed to process image Error:
    Error in Completion Response. Error: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "Resource has been exhausted (e.g. check quota).",
    "status": "RESOURCE_EXHAUSTED"
  }
}

    Please check the status of your model provider API status.
    
18:58:26 - LiteLLM:WARNING: logging_utils.py:113 - `logging_obj` not found - unable to track `llm_api_duration_ms



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



18:58:29 - LiteLLM:WARNING: logging_utils.py:113 - `logging_obj` not found - unable to track `llm_api_duration_ms
18:58:29 - LiteLLM:WARNING: logging_utils.py:113 - `logging_obj` not found - unable to track `llm_api_duration_ms
18:58:29 - LiteLLM:WARNING: logging_utils.py:113 - `logging_obj` not found - unable to track `llm_api_duration_ms
18:58:29 - LiteLLM:WARNING: logging_utils.py:113 - `logging_obj` not found - unable to track `llm_api_duration_ms
18:58:30 - LiteLLM:WARNING: logging_utils.py:113 - `logging_obj` not found - unable to track `llm_api_duration_ms
ERROR:root:Failed to process image Error:expected string or bytes-like object, got 'NoneType'
18:58:30 - LiteLLM:WARNING: logging_utils.py:113 - `logging_obj` not found - unable to track `llm_api_duration_ms
18:58:31 - LiteLLM:WARNING: logging_utils.py:113 - `logging_obj` not found - unable to track `llm_api_duration_ms


report : /Users/nicholasgrundl/projects/ragnostic/data/report/REPORT_ConsultancyOnLargeScaleSubmergedAerobicCultivationProcessDesignNRELGenomatica.pdf
- page count: 27

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

- failed to extract
article : /Users/nicholasgrundl/projects/ragnostic/data/article/REPORT_2003_OptimizePowerConsumptionInAerobicFermenters.pdf
- page count: 4

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

- failed to extract


{'total': {'start': 1737687481.858078,
  'end': 1737687511.871719,
  'elapsed': 0.5},
 'doc_kind': {'start': 1737687511.658027,
  'end': 1737687511.87159,
  'elapsed': 0.0}}